In [2]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('data/clay_export.csv')

# Get all column names
all_columns = df.columns.tolist()

# Extract columns with (CRM) tag
crm_columns = [col.lower() for col in all_columns if '(crm)' in col]

# Extract columns with (export) tag
export_columns = [col.lower() for col in all_columns if '(export)' in col]

# Print the lists
print("CRM Columns:")
print("-" * 50)
for col in crm_columns:
    print(f"  {col}")
print(f"\nTotal CRM columns: {len(crm_columns)}")

print("\n" + "="*50 + "\n")

print("Export Columns:")
print("-" * 50)
for col in export_columns:
    print(f"  {col}")
print(f"\nTotal Export columns: {len(export_columns)}")

# Print a sample of the dataframe shape and first few rows
print("\n" + "="*50 + "\n")
print(f"DataFrame shape: {df.shape}")
print(f"Total columns: {len(df.columns)}")
print(f"Total rows: {len(df)}")


CRM Columns:
--------------------------------------------------
  hubspot id (crm)
  company name (crm)
  company website (crm)
  first name (crm)
  last name (crm)
  job title (crm)
  email (crm)
  phone (crm)
  hubspot owner id (crm)
  linkedin (crm)
  linkedin profile link (crm)
  warmlylinkedinbio (crm)
  linkedin (consolidated) (crm)

Total CRM columns: 13


Export Columns:
--------------------------------------------------
  company name (export)
  company website (export)
  job title (export)
  linkedin (export)
  email (export)
  direct phone (export)
  company phone number (export)
  mobilephone (export)

Total Export columns: 8


DataFrame shape: (733, 75)
Total columns: 75
Total rows: 733


In [5]:
# Test the new column matching functionality
from app.anthropic import ColumnMatcher

# Get the original column names (not lowercase)
original_crm_columns = [col for col in df.columns if '(crm)' in col.lower()]
original_export_columns = [col for col in df.columns if '(export)' in col.lower()]

print("Original CRM columns:")
for col in original_crm_columns:
    print(f"  {col}")

print(f"\nOriginal Export columns:")
for col in original_export_columns:
    print(f"  {col}")

# Initialize the matcher
matcher = ColumnMatcher()

# Match the columns
print("\nMatching columns using LLM...")
result = matcher.match_columns(df, original_crm_columns, original_export_columns)

print("\nColumn Mappings:")
print("=" * 60)
for mapping in result.mappings:
    confidence_str = f"{mapping.confidence:.2f}"
    if mapping.export_column:
        print(f"✓ {mapping.crm_column} → {mapping.export_column} (confidence: {confidence_str})")
        print(f"  Reasoning: {mapping.reasoning}")
        if mapping.is_many_to_one and mapping.additional_crm_columns:
            print(f"  Additional CRM columns: {mapping.additional_crm_columns}")
    else:
        print(f"✗ {mapping.crm_column} → NO MATCH (confidence: {confidence_str})")
        print(f"  Reasoning: {mapping.reasoning}")
    print()

print(f"\nUnmapped CRM columns: {result.unmapped_crm_columns}")
print(f"Unmapped Export columns: {result.unmapped_export_columns}")

if result.notes:
    print(f"\nNotes: {result.notes}")


Original CRM columns:
  HubSpot ID (crm)
  Company Name (crm)
  Company Website (crm)
  First Name (crm)
  Last Name (crm)
  Job Title (crm)
  Email (crm)
  Phone (crm)
  HubSpot Owner ID (crm)
  Linkedin (crm)
  Linkedin Profile Link (crm)
  Warmlylinkedinbio (crm)
  LinkedIn (consolidated) (crm)

Original Export columns:
  Company Name (export)
  Company Website (export)
  Job Title (export)
  LinkedIn (export)
  email (export)
  Direct Phone (export)
  Company Phone Number (export)
  Mobilephone (export)

Matching columns using LLM...

Column Mappings:
✓ Company Name (crm) → Company Name (export) (confidence: 1.00)
  Reasoning: Direct name match with very similar non-null counts (729 vs 690) and matching sample values

✓ Company Website (crm) → Company Website (export) (confidence: 1.00)
  Reasoning: Direct name match with similar non-null counts (666 vs 685) and matching sample values

✓ Job Title (crm) → Job Title (export) (confidence: 1.00)
  Reasoning: Direct name match with sim

In [6]:
# Test the complete enrichment statistics workflow
from app.enrichment.enrichment_calculator import EnrichmentStatisticsCalculator

# Initialize the calculator
calculator = EnrichmentStatisticsCalculator()

# Calculate enrichment statistics using the previous result
print("Calculating enrichment statistics...")
try:
    enrichment_report = calculator.calculate_statistics(
        df=df,
        column_matching_result=result,
        crm_columns=original_crm_columns,
        export_columns=original_export_columns
    )
    print("✓ Enrichment statistics calculated successfully!")
    
    print("\n" + "="*60)
    print("ENRICHMENT REPORT")
    print("="*60)

    print(f"\nBasic Information:")
    print(f"  Total rows: {enrichment_report.total_rows}")
    print(f"  Total CRM columns: {enrichment_report.total_crm_columns}")
    print(f"  Total export columns: {enrichment_report.total_export_columns}")

    print(f"\nGlobal Statistics:")
    print(f"  New columns (export only): {enrichment_report.new_columns_count}")
    print(f"  Many-to-one relationships: {enrichment_report.many_to_one_count}")
    print(f"  Columns reduced by merging: {enrichment_report.columns_reduced_by_merging}")
    print(f"  Records modified: {enrichment_report.records_modified_count}")
    print(f"  Export columns created: {enrichment_report.export_columns_created}")

    print(f"\nColumn-Level Statistics:")
    print("-" * 40)
    for mapping in enrichment_report.column_mappings:
        if mapping.comparison_stats:
            stats = mapping.comparison_stats
            print(f"\n✓ {mapping.crm_column} → {mapping.export_column}")
            print(f"  Confidence: {mapping.confidence:.2f}")
            print(f"  Discarded invalid data: {stats.discarded_invalid_data}")
            print(f"  Added new data: {stats.added_new_data}")
            print(f"  Fixed data: {stats.fixed_data}")
            print(f"  Good data: {stats.good_data}")
            print(f"  Correct % before: {stats.correct_percentage_before:.1f}%")
            print(f"  Correct % after: {stats.correct_percentage_after:.1f}%")
            print(f"  CRM type: {stats.crm_data_type} ({stats.crm_format_count} formats)")
            print(f"  Export type: {stats.export_data_type} ({stats.export_format_count} formats)")
        else:
            print(f"\n✗ {mapping.crm_column} → NO MATCH")
            
    print(f"\nSummary:")
    print(f"  Total mappings processed: {len(enrichment_report.column_mappings)}")
    print(f"  Mappings with statistics: {len([m for m in enrichment_report.column_mappings if m.comparison_stats])}")
    print(f"  Data modification rate: {(enrichment_report.records_modified_count / enrichment_report.total_rows) * 100:.1f}%")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


Calculating enrichment statistics...
✓ Enrichment statistics calculated successfully!

ENRICHMENT REPORT

Basic Information:
  Total rows: 733
  Total CRM columns: 13
  Total export columns: 8

Global Statistics:
  New columns (export only): 2
  Many-to-one relationships: 1
  Columns reduced by merging: 3
  Records modified: 732
  Export columns created: 8

Column-Level Statistics:
----------------------------------------

✓ Company Name (crm) → Company Name (export)
  Confidence: 1.00
  Discarded invalid data: 42
  Added new data: 3
  Fixed data: 264
  Good data: 423
  Correct % before: 99.5%
  Correct % after: 94.1%
  CRM type: string (1 formats)
  Export type: string (1 formats)

✓ Company Website (crm) → Company Website (export)
  Confidence: 1.00
  Discarded invalid data: 37
  Added new data: 56
  Fixed data: 409
  Good data: 220
  Correct % before: 90.9%
  Correct % after: 93.5%
  CRM type: string (1 formats)
  Export type: string (1 formats)

✓ Job Title (crm) → Job Title (expor